In [3]:
import jax
import jax.numpy as jnp
from jax import config
from jax.experimental import jet

config.update("jax_enable_x64", True)  # For better numerical precision

def taylor_expand_ode(vf, initial_value, num_terms):
    """
    Compute Taylor expansion of an ODE solution using JAX's jet.
    
    Args:
        vf: Vector field function that takes state as input
        initial_value: Initial state (should be a JAX array)
        num_terms: Number of terms in the Taylor expansion
    
    Returns:
        List of Taylor coefficients [y0, y1, y2, ..., yn]
        where yi is the i-th Taylor coefficient divided by i!.
    """
    if num_terms == 0:
        return [initial_value]
    
    # Initial term is just the initial value
    taylor_coeffs = [initial_value]
    
    # Compute first derivative (vector field evaluation)
    primals = vf(initial_value)
    taylor_coeffs.append(primals)  # y1 (dy/dt)
    
    factorial = 1  # Keep track of factorial
    
    # Compute higher-order derivatives using jet
    for i in range(1, num_terms - 1):
        factorial *= i + 1  # Update factorial
        
        # Get current coefficients excluding initial value
        series = taylor_coeffs[1:]
        
        # Call jet with appropriate primals and series
        primals_out, series_out = jet.jet(vf, (initial_value,), (series,))
        
        # Normalize the term by factorial
        new_term = series_out[0] / factorial
        
        # Append new coefficient
        taylor_coeffs.append(new_term)
        
    return taylor_coeffs

# Example usage with a simple ODE dy/dt = y (exponential growth)
def exponential_vf(y):
    return y

# Test the implementation
initial_value = jnp.array(1.0)
num_terms = 6

coeffs = taylor_expand_ode(exponential_vf, initial_value, num_terms)

print("Taylor coefficients:")
for i, coeff in enumerate(coeffs):
    print(f"Term {i}: {coeff}")

# Evaluate the Taylor polynomial at t=1.0
def evaluate_taylor_polynomial(coeffs, t):
    result = jnp.zeros_like(coeffs[0])
    for i, coeff in enumerate(coeffs):
        result += coeff * (t ** i)
    return result

t_eval = 1.0
approx_value = evaluate_taylor_polynomial(coeffs, t_eval)
exact_value = jnp.exp(t_eval)

print(f"\nAt t = {t_eval}:")
print(f"Taylor approximation: {approx_value}")
print(f"Exact value: {exact_value}")
print(f"Error: {abs(exact_value - approx_value)}")


Taylor coefficients:
Term 0: 1.0
Term 1: 1.0
Term 2: 0.5
Term 3: 0.16666666666666666
Term 4: 0.041666666666666664
Term 5: 0.008333333333333333

At t = 1.0:
Taylor approximation: 2.7166666666666663
Exact value: 2.718281828459045
Error: 0.0016151617923787498


In [ ]:
import sympy as sp
sp.init_printing()
x_1, x_1_prime, x_2, x_2_prime, R_1,  = sp.symbols('x_1 x_1^{\\prime} R')
display(x_1, x_1_prime)

# Write the taylor expansion of output coordinate x_2 as a function of x_1 and x_1_prime

distance = sp.sqrt(x_1**2 + R_1**2)

In [30]:
import sympy
from sympy import symbols, Function, diff, factorial, Derivative

# -------------------------------------------
# 1) Define our input variables symbolically
# -------------------------------------------
x_in, xprime_in = symbols('x_in xprime_in', real=True)

# ------------------------------------------------------
# 2) Define a generic function F(x_in, xprime_in) in Sympy
#    Note: This is *not* an explicit expression, but a
#    symbolic "placeholder" function.
# ------------------------------------------------------
F = Function('F')(x_in, xprime_in)

# -------------------------------------------------------------
# 3) Define a helper to do the 2D Taylor expansion about (x0,y0)
#    up to orders (nx, ny).
#    Because F is generic, we'll end up with derivative objects
#    like Derivative(F(0,0), x_in).
# -------------------------------------------------------------
def taylor_2d_symbolic(f, x, x0, y, y0, nx, ny):
    """
    Return the two-variable Taylor expansion of a *generic* function
    f(x, y) around (x0, y0), up to 'nx' in (x-x0) and 'ny' in (y-y0).

    Since 'f' is symbolic (no explicit formula), the result will contain
    unevaluated partial derivatives (e.g., Derivative(F(0,0), x)).
    """
    expansion = 0
    # Double-sum for m=0..nx, n=0..ny
    for m in range(nx + 1):
        for n in range(ny + 1):
            # Compute the (m,n)-th partial derivative of f
            # i.e. \partial^{m+n} f / (\partial x^m \partial y^n)
            df = diff(f, (x, m), (y, n))
            
            # Evaluate that derivative at (x0, y0).  For a *generic* F,
            # this remains a 'Derivative(F(0,0), x, ...)' object.
            df_at_0 = df.subs({x: x0, y: y0})
            
            # The standard Taylor term = df_at_0 * ( (x-x0)^m (y-y0)^n ) / (m! n!)
            term = df_at_0 * ( (x - x0)**m * (y - y0)**n ) / ( factorial(m)*factorial(n) )
            expansion += term

    return sympy.simplify(expansion)

# --------------------------------------------
# 4) Perform the Taylor expansion around (0,0)
#    up to first order (nx=1, ny=1) for example
# --------------------------------------------
F_expanded = taylor_2d_symbolic(F, x_in, 0, xprime_in, 0, nx=2, ny=1)

print("F expanded around (x_in=0, xprime_in=0) up to order (1,1):")
print(F_expanded.doit())


F expanded around (x_in=0, xprime_in=0) up to order (1,1):
x_in**2*xprime_in*Subs(Derivative(F(x_in, xprime_in), (x_in, 2), xprime_in), (x_in, xprime_in), (0, 0))/2 + x_in**2*Subs(Derivative(F(x_in, 0), (x_in, 2)), x_in, 0)/2 + x_in*xprime_in*Subs(Derivative(F(x_in, xprime_in), x_in, xprime_in), (xprime_in, x_in), (0, 0)) + x_in*Subs(Derivative(F(x_in, 0), x_in), x_in, 0) + xprime_in*Subs(Derivative(F(0, xprime_in), xprime_in), xprime_in, 0) + F(0, 0)
